## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 
- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 
- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.


In [2]:
import pandas as pd
fi=pd.read_csv('/Users/victormac/PROYECTOS/REPOS/MDS/REPOS-Github/master-data-science/datascience-challengue/dataset_original/searches.csv.bz2'
              , sep='^', low_memory=False, dtype=str, chunksize=1000000)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(fi):
    all_chunks= all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print(i, len(all_chunks))
    
all_chunks.to_csv('searches.no_dup.csv', sep='^', index = False)

0 358999
1 359003
2 359003
3 359003
4 359003
5 359003
6 359003
7 359003
8 359003
9 359003
10 359003
11 359003
12 359003
13 359003
14 359003
15 359003
16 359003
17 359003
18 359003
19 359003
20 359004


In [ ]:
# En lo anterior se ven cuántos duplicados se han aumentado


In [3]:
import pandas as pd
fi=pd.read_csv('/Users/victormac/PROYECTOS/REPOS/MDS/REPOS-Github/master-data-science/datascience-challengue/dataset_original/bookings.csv.bz2'
              , sep='^', low_memory=False, dtype=str, chunksize=1000000)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(fi):
    all_chunks= all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print(i, len(all_chunks))
    
all_chunks.to_csv('bookings.no_dup.csv', sep='^', index = False)

0 1000000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000003
6 1000003
7 1000003
8 1000003
9 1000003
10 1000003


In [16]:
df_bookings = pd.read_csv('/Users/victormac/PROYECTOS/REPOS/MDS/REPOS-Github/master-data-science/datascience-challengue/ejercicios_profesor/bookings.no_dup.csv', 
                          low_memory=False , sep='^', usecols=['dep_port', 'arr_port', 'cre_date           '])


In [22]:
df_searches = pd.read_csv('/Users/victormac/PROYECTOS/REPOS/MDS/REPOS-Github/master-data-science/datascience-challengue/ejercicios_profesor/searches.no_dup.csv', 
                          low_memory=False , sep='^')



In [17]:
df_bookings.shape

(1000003, 3)

In [18]:
df_bookings.head()

,cre_date,dep_port,arr_port
0,2013-02-22 00:00:00,ZRH,LHR
1,2013-03-26 00:00:00,SAL,CLT
2,2013-03-26 00:00:00,SAL,CLT
3,2013-03-26 00:00:00,AKL,SVO
4,2013-03-26 00:00:00,AKL,SVO


In [27]:
#Para quitar espacios
df_bookings.columns = df_bookings.columns.str.strip()
df_bookings.dep_port = df_bookings.dep_port.str.strip()
df_bookings.arr_port = df_bookings.arr_port.str.strip()
# Lo comprobamos
df_bookings.arr_port[0], df_bookings.dep_port[0]

('LHR', 'ZRH')

In [28]:
# Quitar duplicados porque de las tres columnas que hemos seleccionado puede haber duplicados
df_bookings.drop_duplicates(inplace=True)

In [29]:
df_bookings.cre_date[0], df_searches.Date[0]
#tienen diferentes formato de fecha, debería ser igual para hacer el matching

('2013-02-22 00:00:00', '2013-01-01')

In [30]:
# Así sólo cogemos la parte que nos interesa que son las 10 primeras cifras de la fecha de bookings
df_bookings.cre_date = df_bookings.cre_date.str[:10]

In [31]:
df_bookings.cre_date[0], df_searches.Date[0]

('2013-02-22', '2013-01-01')

In [32]:
#Añado la columna de 1
df_bookings['booked'] = 1

In [33]:
df_bookings.head()

,cre_date,dep_port,arr_port,booked
0,2013-02-22,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
5,2013-03-20,DEN,LGA,1
7,2013-03-25,NRT,SIN,1


In [34]:
#Hacemos el merge
bo_se = df_searches.merge(df_bookings,
                         how = 'left',
                         left_on = ['Date', 'Origin', 'Destination'],
                         right_on = ['cre_date', 'dep_port', 'arr_port'])

In [35]:
len(df_searches), len(bo_se)

(359004, 359004)

In [36]:
bo_se.shape

(359004, 49)

In [38]:
bo_se.drop(['cre_date', 'dep_port', 'arr_port'], axis=1, inplace=True)

In [39]:
bo_se.shape

(359004, 46)

In [40]:
bo_se[bo_se['booked'] == 1].head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,booked
27,2013-01-01,18:33:28,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,1.0
40,2013-01-01,06:36:57,FFP,86f167b84e77346849f9439ae87c02a6,SA,DMM,MNL,1.0,2.0,DMM,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,1.0
59,2013-01-01,07:00:38,FQD,e8741eaf2fa2f71f931475d18fa72096,US,ATL,MIA,0.0,1.0,ATL,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SEA,1.0
134,2013-01-01,23:30:44,MPT,5215502d9524c3183f3839b0d9a5e4f9,AU,MEL,SYD,0.0,1.0,MEL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SYD,1.0
172,2013-01-01,18:34:27,MPT,fd4afff0035bec8f8e305d38804c33f6,IN,BOM,JED,1.0,2.0,BOM,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DEL,1.0


In [41]:
bo_se['booked'] = bo_se['booked'].fillna(0)

In [42]:
bo_se.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0.0


In [43]:
bo_se = bo_se.astype({'booked':int})

In [44]:
bo_se.to_csv('searching_with_booking.csv', sep='^', index=False)

In [45]:
pwd

'/Users/victormac/PROYECTOS/REPOS/MDS/REPOS-Github/master-data-science/datascience-challengue/ejercicios_profesor'

In [48]:
#Para ver si existe el archivo
! head ./searching_with_booking.csv

Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice^booked
2013-01-01^20:25:57^MPT^624d8c3ac0b3a7ca03e3c167e0f48327^DE^TXL^AUH^1.0^2.0^TXL^AUH^2013-01-26^D2^^AUH^TXL^2013-02-02^D2^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^FRA^0
2013-01-01^10:15:33^MPT^b0af35b31588dc4ab06d5cf2986e8e02^MD^ATH^MIL^0.0^1.0^ATH^MIL^2013-01-04^^^^^^^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^KIV^0
2013-01-01^18:04:49^MPT^3561a60621de06ab1badc8ca55699ef3^US^ICT^SFO^1.0^2.0^ICT^SFO^2013-08-02^^